T5 and Bart did well. So did Bert + Kmeans with coreference. What happens when we use both.

#Standard installation and imports



In [ ]:
#set up again because restarted runtime
from google.colab import auth
auth.authenticate_user()

project_id = 'test-281700'
!gcloud config set project {project_id}

#download model..restarted runtime
bucket_name = 'spotify_asr_dataset'
!gsutil -m cp -r gs://{bucket_name}/t5-model-3000.zip /content/
!unzip t5-model-3000.zip

Updated property [core/project].
Copying gs://spotify_asr_dataset/t5-model-3000.zip...
| [1/1 files][787.9 MiB/787.9 MiB] 100% Done  39.6 MiB/s ETA 00:00:00           
Operation completed over 1 objects/787.9 MiB.                                    
Archive:  t5-model-3000.zip
   creating: t5-model-3000/
  inflating: t5-model-3000/pytorch_model.bin  
 extracting: t5-model-3000/tokenizer_config.json  
  inflating: t5-model-3000/config.json  
  inflating: t5-model-3000/spiece.model  
  inflating: t5-model-3000/special_tokens_map.json  


In [ ]:
!pip install transformers
!pip install rouge
!pip install neuralcoref==4.0
!pip install spacy==2.1.0 #notebook crashes on other versions
!python -m spacy download en

!git clone https://github.com/paulowoicho/bert-extractive-summarizer.git
!mv bert-extractive-summarizer summarizer

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 15.5MB/s 
     |████████████████████████████████| 3.0MB 17.9MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6a180310e497074d540cd3e215cccf1dd0b768906ad81e22522910dccca0d22e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 296kB 3.1MB/s 
     |████████████████████████████████| 27.7MB 111kB/s 
     |████████████████████████████████| 3.2MB 38.5MB/s 
     |████████████████████████████████| 92kB 10.1MB/s 
     |████████████████████████████████| 2.1MB 34.0MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully un

In [ ]:
import pandas as pd

from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler
from transformers import *
from rouge import Rouge
import spacy
import neuralcoref

100%|██████████| 40155833/40155833 [00:00<00:00, 43455822.07B/s]


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Spanbert + t5 pipeline

In [ ]:
#Spanbert
handler = CoreferenceHandler(greedyness=.4)
custom_config = AutoConfig.from_pretrained("SpanBERT/spanbert-base-cased")
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained("SpanBERT/spanbert-base-cased")
custom_model = AutoModel.from_pretrained("SpanBERT/spanbert-base-cased", config=custom_config)

spanbert_model = Summarizer(custom_model = custom_model, sentence_handler=handler, custom_tokenizer=custom_tokenizer)

def span_bert(transcript, ratio):
  result = spanbert_model(transcript, min_length=60, ratio=ratio)
  return ''.join(result)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#t5
tokenizer = T5Tokenizer.from_pretrained('/content/t5-model-3000')
model = T5ForConditionalGeneration.from_pretrained('/content/t5-model-3000')


def t5_inference(transcript, threshold):
  t5_form = 'summarize: ' + transcript
  tokenized_text = tokenizer.encode(t5_form, return_tensors="pt")
  if len(tokenized_text[0]) > threshold:
    #run out of RAM/crashes on large number of tokens
    revised_text = sent_tokenize(t5_form)
    length = len(revised_text)
    final_text = revised_text[:int(length/2)] #maybe they talk about content in the first half? find proof
    text = ' '.join(final_text)
    return t5_inference(text)
  summary_ids = model.generate(tokenized_text, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output

In [ ]:
def spanbert_t5(transcript, ratio, threshold):
  important_sentences = span_bert(transcript, ratio)
  summary = t5_inference(important_sentences, threshold)
  return summary

#Inference

In [ ]:
import pandas as pd
reference_results = pd.read_csv('150_gold.csv')

In [ ]:
ext_abs = reference_results[['episode_id', 'transcript', 'episode_description']]
ext_abs.head(2)

,episode_id,transcript,episode_description
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Hello everybody. What's going on in this Jess...,If you want to start mastering recruiting whic...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,"Ladies, have you been molested don't be a vic...",Don't be a silent victim of crime. a parody fr...


In [ ]:
ext_abs['auto_summary'] = ext_abs.apply(lambda row: spanbert_t5(row['transcript'], 0.1, 7000), axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ext_abs.iloc[1]['auto_summary']

'Ladies, have you been molested don’t be a victim of crime here are some things you can do when it’s late. Try your best not to be molested by a culprit from a good University who has good academic results and the potential to excel in life. When it’s late try your best not to be molested by a culprit from a good University who has good academic results and the potential to excel in life. If you’ve been molested, try your best not to be'

In [ ]:
ext_abs.iloc[1]['episode_description']

"Don't be a silent victim of crime. a parody from mrbrown.com "

#Evaluation

##spanbert + t5

Just realized I initially did not use Rouge API properly. So I may need to recompute the other results I had gathered, depending on these following results :(. On the other hand, it just means recall and precision scores are switched, so f measure should be the same

In [ ]:
#what i have been doing
rouge = Rouge()
rouge_scores_w = rouge.get_scores(ext_abs['episode_description'], ext_abs['auto_summary'], avg=True)
rouge_scores_w['rouge-1']['f'], rouge_scores_w['rouge-2']['f'], rouge_scores_w['rouge-l']['f']

(0.16932254852889728, 0.0723673729650572, 0.1957123945829847)

In [ ]:
#correct version
rouge_scores_r = rouge.get_scores(ext_abs['auto_summary'], ext_abs['episode_description'], avg=True)
rouge_scores_r['rouge-1']['f'], rouge_scores_r['rouge-2']['f'], rouge_scores_r['rouge-l']['f']

(0.16932254852889728, 0.0723673729650572, 0.19618320623105837)

All is well :). This is better than all extractive techniques but worse than supervised BART and T5. It is better than semi-supervised bart though. However, this depends on the hyperparameter I used (0.1)

#First N and t5 no coreference

In [ ]:
#Forgot first-N again. Rather than rewriting functions, I could make it so that is is a super function that takes
#other functions as parameters

def first_n(transcript, threshold):
  sentences = sent_tokenize(transcript)
  return ' '.join(sentences[:threshold])

def firstn_t5(transcript, limit, threshold):
  sentences = first_n(transcript, limit)
  summary = t5_inference(sentences, threshold)
  return summary

In [ ]:
ext_abs['firstn_summary'] = ext_abs.apply(lambda row: firstn_t5(row['transcript'], 15, 7000), axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
rouge = Rouge()
rouge_scores = rouge.get_scores(ext_abs['firstn_summary'], ext_abs['episode_description'], avg=True)
rouge_scores['rouge-1']['f'], rouge_scores['rouge-2']['f'], rouge_scores['rouge-l']['f']

(0.1909758054021582, 0.0962574068499044, 0.2229754751606156)

Very surprising results! As good as t5 alone! Although no coreference was used

In [ ]:
ext_abs.iloc[10]['firstn_summary']

'Nick and Jeff discuss college football news on the college football news podcast. --- This episode is sponsored by  Anchor: The easiest way to make a podcast. https://anchor.fm/app Support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefoot'

In [ ]:
ext_abs.iloc[10]['episode_description']

'\xa0 Nick Shepkowski sits down with Jeff Feyerer of College Football News, Inside the Pylon and various other football outlets to preview the 2019 season. \xa0 Clemson/Alabama round 5 - who has the upper hand entering 2019? How does the Big Ten stack up to the SEC? \xa0Who are a few underdog picks that could surprise and win their conferences? Predictions, opinions and in depth preview of what the 2019 college football season may bring. \xa0Enjoy!  ---   This episode is sponsored by  · Anchor: The easiest way to make a podcast.  https://anchor.fm/app  Support this podcast: https://anchor.fm/cfnpodcast/support'

In [ ]:
ext_abs.iloc[10]['transcript']

" Hey, it's Nick shop Kowski from the college football news podcast here to tell you about anchor in why I switched to it. One reason. Well, it's free. But beyond that it's awesome had trouble getting my podcast on a previous distributor distributed through, you know, the likes of apple and Spotify and some of the others anchor did that very easily without much question whatsoever. They have creation tools that allow you to record and edit your podcast right from your phone or your computer you can make From your podcast with no minimum listenership. It's everything you need to make a podcast in one place download the free anchor app or go to Anchor dot f m-- to get started.  Welcome back in another edition of the college football news podcast mix-up pow ski with you. Keep you check out for this one, but I welcome it now from a man. You've read his work you've seen as by lines on college football news Pro Football Weekly various different places inside the pile on pile on you podcast. 

#First N and t5 w/ coreference

In [ ]:
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

def coref_resolution(transcript):
  doc = nlp(transcript)
  return doc._.coref_resolved

In [ ]:
def firstn_t5_coref(transcript, limit, threshold):
  sentences = first_n(transcript, limit)
  resolved_sentences = coref_resolution(sentences)
  summary = t5_inference(resolved_sentences, threshold)
  return summary

In [ ]:
ext_abs['firstn_coref_summary'] = ext_abs.apply(lambda row: firstn_t5_coref(row['transcript'], 15, 7000), axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
rouge = Rouge()
rouge_scores = rouge.get_scores(ext_abs['firstn_coref_summary'], ext_abs['episode_description'], avg=True)
rouge_scores['rouge-1']['f'], rouge_scores['rouge-2']['f'], rouge_scores['rouge-l']['f']

(0.19460879330911204, 0.0948191636225971, 0.2267655938612496)

Pretty much the same as without coref, only marginally better. So this says that we definitely do not need most of the transcript, and coref is a nice to have

In [ ]:
ext_abs.iloc[10]['firstn_coref_summary']

'This week, Nick and Jeff discuss college football news on the College Football News Podcast. --- This episode is sponsored by  Anchor: The easiest way to make a podcast. https://anchor.fm/app Support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/collegefootballnews/support this podcast: https://anchor.fm/'

In [ ]:
ext_abs.iloc[4]['firstn_coref_summary']

"This is the first episode of The Other School Podcast. I talk about what it means to be a teenager and why you should keep listening! Thank you so much for your patience, thank you so much for tuning in and thanks for listening! Today's session is going to be a bit different from the norm. We will go over what they don't teach you at school and how to learn these essential lessons. From teenage Finance earning money to personality Improvement. Please leave us a review on Instagram or Twitter @UncleSharif Sharma Facebook Group: https://www.instagram.com/uncleSharif Sharma YouTube Channel: https://www.youtube.com/uncleSharif Sharma"

In [ ]:
ext_abs.iloc[4]['episode_description']

'Introduction | Show Formalities | What to Expect This show is all about real learning.Teaching youn what they dont teach you at school. Everything you struggle with as a teenager.And the basics of what should be taught at school.for more information email @unschoolpodcast@gmail.com Host ; Ankush Sharma\xa0 '